In [1]:
#@title Aperte o botão ao lado para executar o programa

print("GERADOR DE RELATÓRIO PARA BOLETOS DE GRU DA INOVA UNICAMP.\n")

print("\nAguarde a instalação das bibliotecas necessárias.\n")

import pandas as pd
# !pip install -q pdfminer3
# !pip install -q XlsxWriter
from datetime import date, timedelta
from pdfminer3.layout import LAParams
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import TextConverter
import io
import os
import sys
import re

####### FUNCTIONS AND CLASSES #######

def parse_txt(text,start,end):
    i = text.find(start) + len(start)
    f = text.find(end)
    txt = text[i:f]
    return txt

class modelo_febraban_BB:
    def __init__(self, nr_codb):
        self.nr_codb = nr_codb
        
    def cod_manual(self): # remove espaços do número de digitação manual de 47 digitos (44 + digitos verificadores dos campos)
        return self.nr_codb.replace('.','').replace(' ','')
    
    def cod_leitora(self): # converte o número de digitação manual no código de barras para a leitora com 44 digitos
        return self.cod_manual()[0:3] + self.cod_manual()[3:4] + self.cod_manual()[32:33] + self.cod_manual()[33:37] + self.cod_manual()[37:47] + self.cod_manual()[4:9] + self.cod_manual()[10:20] + self.cod_manual()[21:31]
    
    def elements(self):
        elements = []
        elements.append(self.cod_manual()[0:3]) # A - numero do banco
        elements.append(self.cod_manual()[3:4]) # B - digito moeda real = 9
        elements.append(self.cod_manual()[4:9]) # C - posições 20 a 24 do código de barras (zeros)
        elements.append(self.cod_manual()[9:10]) # X - digito verificador campo 1
        elements.append(self.cod_manual()[10:20]) # D - Posições 25 a 34 do código de barras (convênio fornecido pelo banco) 
        elements.append(self.cod_manual()[20:21]) # Y - digito verificador campo 2
        elements.append(self.cod_manual()[21:31]) # E - Posições 35 a 44 do código de barras (comp nosso numero/cart mod cobr)
        elements.append(self.cod_manual()[31:32]) # Z - digito verificador campo 3
        elements.append(self.cod_manual()[32:33]) # K - digito verificador do código de barras deve ser diferente de 0
        elements.append(self.cod_manual()[33:37]) # U - Fator de Vencimento (cálculo conforme anexo VI)
        elements.append(self.cod_manual()[37:47]) # V - Valor do boleto de pagamento (com duas casas decimais, sem ponto e vírgula. Em caso de moeda variável, informar zeros)
        return elements
    
    def banco(self): # retorna o banco do boleto
        if self.cod_manual()[0:3] == "001": # GRUs são sempre do BB
            return "Banco do Brasil"
        else: # preciso de uma lista com códigos de banco
            return "Outro banco"
        
    def moeda(self):
        if self.cod_manual()[3:4] == "9":
            return "Real (R$)"
        else: # preciso de uma lista com códigos de banco
            return "Moeda estrangeira"
    
    def vencimento(self): # determina o vencimento pelo fator de vencimento do código de barras
        fator_vencimento = self.cod_manual()[33:37] # fator de vencimento do código de barras
        vencimento = date(2000, 7, 3) + timedelta(days=int(fator_vencimento)-1000) # primeira data 03/07/2000 com fator 1000 e última data 21/02/2025 com fator 9999, e reinicia os fatores de novo 
        return vencimento.strftime('%d/%m/%Y')
    
    def valor(self):
        return int(self.cod_manual()[37:47])/100
    
    def modulo_11(self): # checa o digito verificador do codigo de barras de acordo com o modulo 11
        nr = self.cod_leitora()
        p = len(nr) # posição
        sum_dv = 0
        k = 2 # multiplicador
        for i in range(44):
            if p != 5:
                if 2<=k<=9:
                    sum_dv = sum_dv + int(nr[p-1:p])*k
                if k>9:
                    k = 2
                    sum_dv = sum_dv + int(nr[p-1:p])*k
            if p == 5:
                k-=1
            k+=1
            p-=1
        
        dv = 11 - sum_dv%11
        if dv == 0 or dv == 10 or dv == 11:
            return 1
        if dv != 0 or dv != 10 or dv != 11:
            return dv
    
    def modulo_10(self): # checa o digito dos campos 1, 2 e 3 de acordo com o modulo 10
        i = 0
        j = 9
        dv_f = []
        for x in range(3): # verifica um campo de cada vez
            nr = self.cod_manual()[i:j]
            p = 0
            sum_dv = 0
            for i in range(int(len(nr))): # verifica o campo i
                if i%2 == 1:
                    if len(nr) == 9:
                        k = 1
                    if len(nr) == 10:
                        k = 2
                if i%2 == 0:
                    if len(nr) == 9:
                        k = 2
                    if len(nr) == 10:
                        k = 1
                if (int(nr[p:p+1])*k) > 9:
                    sum_dv = sum_dv + (int(nr[p:p+1])*k)//10 + (int(nr[p:p+1])*k)%10
                if (int(nr[p:p+1])*k) <= 9:
                    sum_dv = sum_dv + int(nr[p:p+1])*k
                p+=1
            
            dv = 10 - sum_dv%10
            if dv == 10:
                dv_f.append(0)
            if dv != 10:
                dv_f.append(dv)
            i = j + 1
            j = i + 10
            
        return dv_f
    
class pdf_parse:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
            
    def pdf_to_txt(self): # source: https://stackoverflow.com/questions/56494070/how-to-use-pdfminer-six-with-python-3
        resource_manager = PDFResourceManager()
        fake_file_handle = io.StringIO()
        converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
        page_interpreter = PDFPageInterpreter(resource_manager, converter)

        with open(self.pdf_path, 'rb') as fh:
            for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
                page_interpreter.process_page(page)

            text = fake_file_handle.getvalue()

        # close open handles
        converter.close()
        fake_file_handle.close()

        return text
    
print("Concluído!")

GERADOR DE RELATÓRIO PARA BOLETOS DE GRU DA INOVA UNICAMP.


Aguarde a instalação das bibliotecas necessárias.

Concluído!


In [2]:
# # Apagar arquivos pré-existentes para evitar erro no Google Colab

# !rm /content/*.pdf # apagando os arquivos pdf
# !rm /content/*.xlsx # apagando os arquivos xlsx

# print("Concluído.")

In [3]:
# ###### Para uso no Google Colab, comente se for usar localmente

# print('\033[1m' + "\nCLIQUE EM UPLOAD E SELECIONE OS ARQUIVOS.\n" + '\033[0m')

# print('\033[1m' + "\nIMPORTANTE! A soma de todos os arquivos não pode ultrapassar 70GB!\n" + '\033[0m')

# files.upload()

In [7]:
####### START CODE #######

files_list = []
with os.scandir() as files:
    for entry in files:
        if entry.name.endswith('.pdf') and entry.is_file():
            files_list.append(entry.name)
            
data = files_list[0].split("_")[-1].split(".")[0]

row = 0
# report_df = pd.DataFrame(columns=['Arquivo', 'Código de Barra', 'Valor', 'Vencimento', 'Processo', 'Serviço'])
report_df = pd.DataFrame(columns=['Tecnologia', 'PI', 'Revista', 'Despacho', 'Valor', 'Serviço'])
for i in range(len(files_list)):
    file = files_list[i]
    #print(file) # imprimir o nome do arquivo
    text = pdf_parse(file).pdf_to_txt()
            
    # extrair codigo de barra
    scan_cod = parse_txt(text,"Pessoa Jurídica.\n\n","\n\nUNIVERSIDADE")
    
    if scan_cod == "":
        check_cb = False
        report_df.loc[i] = [file,"","","","","Erro! Verifique o arquivo!"]
        row=i
    if scan_cod != "":
        check_cb = True
        
    # checar o código de barras
    if check_cb == True:
        scan_cod2 = modelo_febraban_BB(scan_cod).cod_manual()
        flag1 = int(scan_cod2[9:10]) == modelo_febraban_BB(scan_cod).modulo_10()[0]
        flag2 = int(scan_cod2[20:21]) == modelo_febraban_BB(scan_cod).modulo_10()[1]
        flag3 = int(scan_cod2[31:32]) == modelo_febraban_BB(scan_cod).modulo_10()[2]
        flag4 = int(scan_cod2[32:33]) == modelo_febraban_BB(scan_cod).modulo_11()
                
        if flag1 == flag2 == flag3 == flag4 == True:
            valor = modelo_febraban_BB(scan_cod2).valor()

            string1 = file.split("_")[0]
            if string1[:2] == "BR" or string1[:2] == "PI" or string1[:2] == "MU":
                tecnologia = file.split("_")[0]
                if file.find("RPI") != -1:
                    pos = file.find("RPI")
                    revista = file[pos:pos+7]
                if file.find("RPI") == -1:
                    revista = ""
                despacho = str(re.findall('_([0-9]+.*?)_', file))[2:-2]
                if text.find("Processo") != -1:
                    processo = parse_txt(text,"\nProcesso: "," \nServiço").replace('\n','')
                processo = parse_txt(text,"\nProcesso: "," \nServiço").replace('\n','')
                if processo[0].isdigit() == True:
                    processo = "BR " + processo
                if text.find("Processo") == -1:
                    processo = tecnologia
                
            if string1[:2] != "BR" and string1[:2] != "PI" and string1[:2] != "MU":
                tecnologia = file.split("_")[0] + "_" + file.split("_")[1]
                if len(file.split("_")) == 7:
                    if file.find("RPI") != -1:
                        pos = file.find("RPI")
                        revista = file[pos:pos+7]
                    if file.find("RPI") == -1:
                        revista = ""
                        despacho = str(re.findall('_([0-9]+.*?)_', file))[2:-2]
                    despacho = str(re.findall('_([0-9]+.*?)_', file))[2:-2]
                    
                if len(file.split("_")) == 6:
                    if file.find("RPI") != -1:
                        pos = file.find("RPI")
                        revista = file[pos:pos+7]
                    if file.find("RPI") == -1:
                        revista = ""
                        despacho = str(re.findall('_([0-9]+.*?)_', file))[2:-2]
                    despacho = str(re.findall('_([0-9]+.*?)_', file))[2:-2]
                    
                if text.find("Processo") != -1:
                    processo = parse_txt(text,"\nProcesso: "," \nServiço").replace('\n','')
                    if processo[0].isdigit() == True:
                        processo = "BR " + processo
                if text.find("Processo") == -1:
                    processo = ""
             
            vencimento = modelo_febraban_BB(scan_cod2).vencimento()
            if text.find("Serviço") != -1:
                servico = parse_txt(text," \nServiço: "," \n\nUNIVERSIDADE").replace('\n','')
            if text.find("Serviço") == -1:
                servico = ""
            #report_df.loc[i] = [file,scan_cod,valor,vencimento,processo,servico]
            report_df.loc[i] = [tecnologia,processo,revista,despacho,valor,servico]
            
        if flag1 == flag2 == flag3 == flag4 == False:            
            report_df.loc[i] = [file,"","","","","Erro! Verifique o arquivo!"]
            row=i
            
        row = i

row+=2        
soma = "=SUM(E2:E" + str(row) + ")"
soma

# Formatando o campo data
# report_df['Vencimento'] = pd.to_datetime(report_df['Vencimento']).dt.date
            
# Configurando o arquivo em excel

# Create a Pandas Excel writer using XlsxWriter as the engine.
# Also set the default datetime and date formats.

xlsx_name = "CARTA_DGA_" + str(data) + "_" + "Anexo.xlsx"

writer = pd.ExcelWriter(xlsx_name, engine='xlsxwriter', date_format='dd/mm/yyyy')

# Convert the dataframe to an XlsxWriter Excel object.
report_df.to_excel(writer, sheet_name='Sheet1', index=False)

# Get the xlsxwriter workbook and worksheet objects in order to set the column
# widths, to make the dates clearer.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']
worksheet.write(row, 4, soma)

# Add a number format for cells with money.
need_download = workbook.add_format()
need_download.set_bg_color('yellow')
need_download.set_font_color('red')
money_fmt = workbook.add_format({'align': 'center', 'num_format': 'R$ #.##0'})
align_fmt = workbook.add_format({'align': 'center'})
worksheet.set_column('A:A', 20)
worksheet.set_column('B:B', 20, align_fmt)
worksheet.set_column('C:C', 15)
worksheet.set_column('D:D', 10)
worksheet.set_column('E:E', 20, money_fmt)
worksheet.set_column('F:F', 60)

worksheet.conditional_format('F2:F300', {'type': 'cell',
                                        'criteria': 'equal to',
                                        'value': '"Erro! Verifique o arquivo!"',
                                        'format': need_download})

# Close the Pandas Excel writer and output the Excel file.
writer.save()

print('\033[1m' + "\nCONCLUÍDO!" + '\033[0m')


CONCLUÍDO!


In [ ]:
# ### Para uso no Google Colab

# !rm /content/*.pdf # apagando os arquivos pdf

# print('\033[1m' + "\nConcluído! Faça o download do arquivo no menu ao lado." + '\033[0m')